# 机器人画家

In [18]:
import numpy as np
from pydrake.all import (
    AbstractValue,
    ConstantVectorSource,
    DiagramBuilder,
    LeafSystem,
    PiecewisePose,
    RigidTransform,
    RotationMatrix,
    Simulator,
    StartMeshcat,
)

from manipulation import running_as_notebook
from manipulation.exercises.grader import Grader
from manipulation.exercises.pick.test_robot_painter import TestRobotPainter
from manipulation.meshcat_utils import AddMeshcatTriad
from manipulation.scenarios import AddIiwaDifferentialIK
from manipulation.station import LoadScenario, MakeHardwareStation

In [19]:
# 启动可视化器。
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7001


在下面的单元格中，我们提供了一个包装类，用于隐藏 Drake 中的部分实现细节。您不需要理解它是如何工作的。

In [20]:
class PoseTrajectorySource(LeafSystem):
    def __init__(self, pose_trajectory):
        LeafSystem.__init__(self)
        self._pose_trajectory = pose_trajectory
        self.DeclareAbstractOutputPort(
            "pose", lambda: AbstractValue.Make(RigidTransform()), self.CalcPose
        )

    def CalcPose(self, context, output):
        output.set_value(self._pose_trajectory.GetPose(context.get_time()))


class IIWA_Painter:
    def __init__(self, traj=None):
        builder = DiagramBuilder()
        scenario_data = """
        directives:
        - add_directives:
            file: package://manipulation/clutter.dmd.yaml
        model_drivers:
            iiwa: !IiwaDriver
                control_mode: position_only
                hand_model_name: wsg
            wsg: !SchunkWsgDriver {}
        """
        scenario = LoadScenario(data=scenario_data)
        self.station = builder.AddSystem(MakeHardwareStation(scenario, meshcat=meshcat))
        self.plant = self.station.GetSubsystemByName("plant")
        # 移除手腕关节的关节限制。
        self.plant.GetJointByName("iiwa_joint_7").set_position_limits(
            [-np.inf], [np.inf]
        )
        controller_plant = self.station.GetSubsystemByName(
            "iiwa_controller_plant_pointer_system",
        ).get()

        # 可选添加轨迹源
        if traj is not None:
            traj_source = builder.AddSystem(PoseTrajectorySource(traj))
            self.controller = AddIiwaDifferentialIK(
                builder,
                controller_plant,
                frame=controller_plant.GetFrameByName("body"),
            )
            builder.Connect(
                traj_source.get_output_port(),
                self.controller.get_input_port(0),
            )
            builder.Connect(
                self.station.GetOutputPort("iiwa.state_estimated"),
                self.controller.GetInputPort("robot_state"),
            )

            builder.Connect(
                self.controller.get_output_port(),
                self.station.GetInputPort("iiwa.position"),
            )
        else:
            iiwa_position = builder.AddSystem(ConstantVectorSource(np.zeros(7)))
            builder.Connect(
                iiwa_position.get_output_port(),
                self.station.GetInputPort("iiwa.position"),
            )

        wsg_position = builder.AddSystem(ConstantVectorSource([0.1]))
        builder.Connect(
            wsg_position.get_output_port(),
            self.station.GetInputPort("wsg.position"),
        )

        self.diagram = builder.Build()
        self.gripper_frame = self.plant.GetFrameByName("body")
        self.world_frame = self.plant.world_frame()

        context = self.CreateDefaultContext()
        self.diagram.ForcedPublish(context)

    def visualize_frame(self, name, X_WF, length=0.15, radius=0.006):
        """
        可视化不附加到现有物体的虚构框架

        输入：
            name: 框架的名称 (str)
            X_WF: 从框架 F 到世界的 RigidTransform。

        名称已存在的框架将被新框架覆盖
        """
        AddMeshcatTriad(
            meshcat, "painter/" + name, length=length, radius=radius, X_PT=X_WF
        )

    def CreateDefaultContext(self):
        context = self.diagram.CreateDefaultContext()
        plant_context = self.diagram.GetMutableSubsystemContext(self.plant, context)

        # 提供初始状态
        q0 = np.array(
            [
                1.40666193e-05,
                1.56461165e-01,
                -3.82761069e-05,
                -1.32296976e00,
                -6.29097287e-06,
                1.61181157e00,
                -2.66900985e-05,
            ]
        )
        # 设置 kuka 手臂的关节位置
        iiwa = self.plant.GetModelInstanceByName("iiwa")
        self.plant.SetPositions(plant_context, iiwa, q0)
        self.plant.SetVelocities(plant_context, iiwa, np.zeros(7))
        wsg = self.plant.GetModelInstanceByName("wsg")
        self.plant.SetPositions(plant_context, wsg, [-0.05, 0.05])
        self.plant.SetVelocities(plant_context, wsg, [0, 0])

        return context

    def get_X_WG(self, context=None):
        if not context:
            context = self.CreateDefaultContext()
        plant_context = self.plant.GetMyMutableContextFromRoot(context)
        X_WG = self.plant.CalcRelativeTransform(
            plant_context, frame_A=self.world_frame, frame_B=self.gripper_frame
        )
        return X_WG

    def paint(self, sim_duration=20.0):
        context = self.CreateDefaultContext()
        simulator = Simulator(self.diagram, context)

        meshcat.StartRecording(set_visualizations_while_recording=False)
        duration = sim_duration if running_as_notebook else 0.01
        simulator.AdvanceTo(duration)
        meshcat.PublishRecording()

# 问题描述
在讲座中，我们学习了空间变换的基础知识。对于这个练习，您将让 iiwa 手臂通过计算和插值关键帧来“绘制”一个圆形平面轨迹，就像我们在讲座中看到的那样

**这些是练习的主要步骤：**
1. 为 Iiwa 手臂设计并实现一个圆形轨迹以跟随。
2. 观察并反思微分 IK 控制器。

# 圆形轨迹

在这个练习中，您将为 iiwa 手臂设计一个像下面这样的圆形平面轨迹，让它像机器人一样在空中绘画！为此，我们将遵循课堂上显示的相同程序：

(1) 计算圆形轨迹的关键帧

(2) 从关键帧构造插值轨迹

<img src="https://raw.githubusercontent.com/RussTedrake/manipulation/master/book/figures/exercises/robot_painter_circle.png" width="700">

上图中的 x 和 y 轴来自世界坐标系。

<img src="https://raw.githubusercontent.com/RussTedrake/manipulation/master/book/figures/exercises/robot_painter_screenshot.png" width="700">

上面的截图可视化了圆形轨迹的关键帧。关键帧说明了夹持器在轨迹沿途不同时间步长的世界坐标系中的姿势。首先，您应该从上面的可视化中注意到夹持器坐标系不同于世界坐标系。特别是，夹持器坐标系的 +y 轴垂直向下，+z 轴向后指。这对这个练习很重要。

圆形轨迹中心的刚性变换以及圆的半径定义如下。用语言来说，我们希望我们的手臂围绕世界坐标系的 +z 轴逆时针旋转。此外，我们希望夹持器坐标系的 +z 轴始终指向圆的中心。

In [21]:
# 定义中心和半径
radius = 0.1
p0 = [0.45, 0.0, 0.4]
R0 = RotationMatrix(np.array([[0, 1, 0], [0, 0, -1], [-1, 0, 0]]).T)
X_WCenter = RigidTransform(R0, p0)

num_key_frames = 100
"""
您可以使用不同的 thetas，只要您的轨迹从上面的起始框架开始，
并且您的旋转在世界框架的 +z 轴上是正的
thetas = np.linspace(0, 2*np.pi, num_key_frames)
"""
thetas = np.linspace(0, 2 * np.pi, num_key_frames)

painter = IIWA_Painter()

我们在本笔记本的顶部提供了 `IIWA_painter` 类，以帮助您抽象化 Drake 中的部分实现细节。您可能会发现 `visualize_frame` 方法有助于可视化刚性变换。下面的单元格首先计算当前夹持器姿势的刚性变换，然后在 meshcat 中绘制该姿势的框架。请注意，这里绘制的框架不附加到场景中的任何物体上。它们仅用于可视化。

In [22]:
X_WG = painter.get_X_WG()
painter.visualize_frame("gripper_current", X_WG)

最后，您可以通过 `MakeXRotation`、`MakeYRotation` 和 `MakeZRotation` 方法组合任意旋转。它们的名字很自解释。

In [23]:
RotationMatrix.MakeYRotation(np.pi / 6.0)

RotationMatrix([
  [0.8660254037844387, 0.0, 0.49999999999999994],
  [0.0, 1.0, 0.0],
  [-0.49999999999999994, 0.0, 0.8660254037844387],
])

**下面，您的工作是完成 `compose_circular_key_frames` 方法，给定圆的中心、期望的半径以及关键帧围绕圆中心的插值旋转角度**

In [24]:
def compose_circular_key_frames(thetas, X_WCenter, radius):
    """
    返回：RigidTransforms 的列表
    """
    key_frame_poses_in_world = []
    
    # 获取圆心的位置和方向
    p_center = X_WCenter.translation()
    R_center = X_WCenter.rotation()
    
    for theta in thetas:
        # 在圆心坐标系中计算圆上的位置
        # 根据圆心坐标系的定义：
        # x轴 -> 世界y轴, y轴 -> 世界-z轴, z轴 -> 世界-x轴
        # 我们要在x-y平面上画圆
        x_circle = radius * np.cos(theta)
        y_circle = radius * np.sin(theta)
        z_circle = 0.0
        
        # 在圆心坐标系中的位置
        p_circle_center = np.array([x_circle, y_circle, z_circle])
        
        # 转换到世界坐标系
        p_world = p_center + R_center @ p_circle_center
        
        # 计算夹持器的方向：+z轴指向圆心
        z_axis = (p_center - p_world)
        z_axis = z_axis / np.linalg.norm(z_axis)  # 归一化
        
        # 计算切向量 - 在圆心坐标系中的切向量
        # 在圆心坐标系中，切向量为 (-sin(theta), cos(theta), 0)
        tangent_center = np.array([-np.sin(theta), np.cos(theta), 0.0])
        # 转换到世界坐标系
        x_axis = R_center @ tangent_center
        x_axis = x_axis / np.linalg.norm(x_axis)
        
        # 计算y轴（右手坐标系）
        y_axis = np.cross(z_axis, x_axis)
        y_axis = y_axis / np.linalg.norm(y_axis)
        
        # 构建旋转矩阵
        R_gripper = RotationMatrix(np.column_stack([x_axis, y_axis, z_axis]))
        
        # 创建刚体变换
        this_pose = RigidTransform(R_gripper, p_world)
        key_frame_poses_in_world.append(this_pose)

    return key_frame_poses_in_world

In [25]:
def visualize_key_frames(frame_poses):
    for i, pose in enumerate(frame_poses):
        painter.visualize_frame("frame_{}".format(i), pose, length=0.05)


key_frame_poses = compose_circular_key_frames(thetas, X_WCenter, radius)
visualize_key_frames(key_frame_poses)

## 构造轨迹

现在我们使用 `PiecewisePose` 构造轨迹来插值关键帧的位置和方向。

In [26]:
X_WGinit = painter.get_X_WG()
total_time = 20
key_frame_poses = [X_WGinit] + compose_circular_key_frames(thetas, X_WCenter, radius)
times = np.linspace(0, total_time, num_key_frames + 1)
traj = PiecewisePose.MakeLinear(times, key_frame_poses)

现在您应该能够可视化圆形绘画的执行。使用它来确认夹持器逆时针移动，跟随场景中之前绘制的关键帧。

In [27]:
painter = IIWA_Painter(traj)
painter.paint(sim_duration=total_time)

## 反思

**查看上面 `IIWA_Painter` 类的构造函数。请注意，我们使用 `AddIiwaDifferentialIK` 作为我们的控制器。内部，它使用系统 [DifferentialInverseKinematicsIntegrator](https://drake.mit.edu/pydrake/pydrake.multibody.inverse_kinematics.html#pydrake.multibody.inverse_kinematics.DifferentialInverseKinematicsIntegrator)。阅读文档并推理它是如何工作的，并回答以下三个问题（每题 1-2 句话就足够）。**
1. 从文档：`DifferentialInverseKinematicsIntegrator` 集成对 `DoDifferentialInverseKinematics` 的连续调用。为什么集成是必要的？
2. 为什么设置积分器的初始状态很重要？
3. 我们的代码如何设置积分器的初始状态？查看 `IIWA_Painter` 类的构造函数。

### 您的答案

在这里回答问题，并复制粘贴到 Gradescope 的“书面提交”部分！

## 这个笔记本将如何评分？

如果您参加了课程，这个笔记本将使用 [Gradescope](www.gradescope.com) 评分。您应该在 Piazza 的公告中获得了注册码。

对于这个作业的提交，您必须做两件事。
- 下载并提交笔记本 `robot_painter.ipynb` 到 Gradescope 的笔记本提交部分，以及其他问题的笔记本。

我们将评估笔记本中的本地函数，看看函数是否按照我们的预期运行。对于这个练习，评分标准如下：
- [4.0 分] `compose_circular_key_frames` 根据要求正确
- [3.0 分] 书面问题的合理答案

In [28]:
Grader.grade_output([TestRobotPainter], [locals()], "results.json")
Grader.print_test_results("results.json")

Total score is 4/4.

Score for compose_circular_key_frames is 4/4.
